Finally, we write the code file itself.  Most of the text below will appear literally in the file.  Statements in single curly braces will be substituted by the `format` statement at the end of the string; double curly braces will just appear as literal braces in the output.

In the following, the ODE system directly integrates the quaternion rotors describing the spin directions and orbital angular velocity vector.  This allows us to reduce the number of variables in the system and automatically satisfy certain constraints, which should lead to better numerical behavior.

Note that in this formulation, `Phi` is a completely unnecessary variable.  It doesn't need to be computed because it is implict in the definition of the frame.  It is calculated here simply because it is easy to do, and might be handy to have around.  Of course, it could be derived from knowledge of the frame alone.

In [1]:
# Always run this first
# NOTE: Do not define new basic variables in this notebook;
#       define them in Variables_Q.ipynb.  Use this notebook
#       to define new expressions built from those variables.

from __future__ import division # This needs to be here, even though it's in Variables.py
import sys
sys.path.insert(0, '..') # Look for modules in directory above this one
execfile('../Utilities/ExecNotebook.ipy')
UseQuaternions = True
from sympy import N
from sympy import Rational as frac # Rename for similarity to latex
execnotebook('../PNTerms/Variables_Q.ipynb')
from Utilities import CodeOutput
from textwrap import indent

/home/dongze/.local/lib/python3.8/site-packages/IPython/nbformat.py:12: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  warn("The `IPython.nbformat` package has been deprecated since IPython 4.0. "
/home/dongze/.local/lib/python3.8/site-packages/nbformat/current.py:15: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  warnings.warn("""nbformat.current is deprecated.


In [2]:
HighestOrder=frac(8,2)
with open('PNEvolution.py', 'w') as f :
    f.write("""# File produced automatically by PNCodeGen.ipynb
from scipy.integrate import solve_ivp
import numpy as np
from numpy import dot, cross, log, sqrt, pi
from numpy import euler_gamma as EulerGamma
from numba import jit, njit, float64, boolean
from numba.experimental import jitclass
from scipy.interpolate import InterpolatedUnivariateSpline as Spline
from scipy.special import zeta
import quaternionic

qmul = njit(quaternionic.algebra.multiply)
qexp=njit(quaternionic.algebra.exp)
qconj=njit(quaternionic.algebra.conj)
qinverse=njit(quaternionic.algebra.reciprocal)

@njit(cache=True)
def mul(A,B):
    C=np.empty(4)
    qmul(A,B,C)
    return C
    
@njit(cache=True)
def exp(A):
    B=np.empty(4)
    qexp(A,B)
    return B
    
@njit(cache=True)
def conjugate(A):
    B=np.empty(4)
    qconj(A,B)
    return B
    
@njit(cache=True)
def inverse(A):
    B=np.empty(4)
    qinverse(A,B)
    return B

@njit(cache=True)
def FrameFromAngularVelocity_2D_Integrand(rfrak_x, rfrak_y, Omega):
    rfrakMag = np.sqrt(rfrak_x*rfrak_x+rfrak_y*rfrak_y)
    rfrakDot_x = Omega[0]/2.0
    rfrakDot_y = Omega[1]/2.0
    if np.abs(np.sin(rfrakMag)) > 1e-12 and np.abs(np.cos(rfrakMag)) > 1e-12:
        omega_v = (Omega[0]*(-rfrak_y/rfrakMag)+Omega[1]*(rfrak_x/rfrakMag))*np.tan(rfrakMag)-Omega[2]
        Omega[0] += -omega_v*np.sin(2*rfrakMag)*(-rfrak_y/rfrakMag)
        Omega[1] += -omega_v*np.sin(2*rfrakMag)*(rfrak_x/rfrakMag)
        Omega[2] +=  omega_v*np.cos(2*rfrakMag)
        dotTerm = (rfrak_x*Omega[0]+rfrak_y*Omega[1])/(rfrakMag*rfrakMag)
        cotTerm = rfrakMag/(2*np.tan(rfrakMag))
        rfrakDot_x = (Omega[0] - rfrak_x*dotTerm)*cotTerm + rfrak_x*dotTerm/2. - 0.5*Omega[2]*rfrak_y
        rfrakDot_y = (Omega[1] - rfrak_y*dotTerm)*cotTerm + rfrak_y*dotTerm/2. + 0.5*Omega[2]*rfrak_x
    return rfrakDot_x, rfrakDot_y

@njit(cache=True)
def FrameFromAngularVelocityIntegrand(rfrak, Omega):
    rfrakMag = np.sqrt(rfrak[0] * rfrak[0] + rfrak[1] * rfrak[1] + rfrak[2] * rfrak[2])
    OmegaMag = np.sqrt(Omega[0] * Omega[0] + Omega[1] * Omega[1] + Omega[2] * Omega[2])
    # If the matrix is really close to the identity, return
    if rfrakMag < 1e-12*OmegaMag:
        return np.array([Omega[0] / 2.0, Omega[1] / 2.0, Omega[2] / 2.0])
    # If the matrix is really close to singular, it's equivalent to the identity, so return
    if np.abs(np.sin(rfrakMag)) < 1e-12:
        return np.array([Omega[0] / 2.0, Omega[1] / 2.0, Omega[2] / 2.0])
    OmegaOver2 = np.array([Omega[0] / 2.0, Omega[1] / 2.0, Omega[2] / 2.0])
    rfrakHat = np.array([rfrak[0] / rfrakMag, rfrak[1] / rfrakMag, rfrak[2] / rfrakMag])
    return ((OmegaOver2 - rfrakHat * np.dot(rfrakHat, OmegaOver2)) * (rfrakMag / np.tan(rfrakMag))
        + rfrakHat * np.dot(rfrakHat, OmegaOver2) + np.cross(OmegaOver2, rfrak))  
""")
    PNOrbitalEvolutionOrder=HighestOrder
    PNOrbitalEvolutionOrderString=str(N(PNOrbitalEvolutionOrder, 2)).replace('.','p')
    %cd -q ../PNTerms
    execnotebook('OrbitalEvolution.ipynb')
    ExpressionsForMemberFunctions = CodeOutput.CodeConstructor(PNVariables, PrecessionVelocities)
    execnotebook('AngularMomentum.ipynb')
    %cd -q -
    PrecessionVelocities.AddDerivedVariable('OrbitalAngularMomentum',
        AngularMomentumExpression(PNOrder=PNOrbitalEvolutionOrder),
        datatype=ellHat.datatype)
    CodeConstructor.AddDependencies(PrecessionVelocities)
    f.write("""
ConsSpec=[{ConsSpec},('EvolveSpin1',boolean),('EvolveSpin2',boolean)]
@jitclass(ConsSpec)
class Cons:
    def __init__(self,{ConsArgu},EvolveSpin1,EvolveSpin2):
{ConsInit}
        self.EvolveSpin1=EvolveSpin1
        self.EvolveSpin2=EvolveSpin2

VarsSpec=[{VarsSpec}]
@jitclass(VarsSpec)
class Vars:
    def __init__(self,{VarsArgu}):
{VarsInit}

@njit(cache=True)
def Initialization(Cons, {InputArguments}): 
{Initializations}
    Cons.EvolveSpin1=np.linalg.norm(mul(Cons.S_chi1,conjugate(Cons.S_chi1)))>1e-8
    Cons.EvolveSpin2=np.linalg.norm(mul(Cons.S_chi2,conjugate(Cons.S_chi2)))>1e-8
""".format(ConsSpec=CodeConstructor.ConsSpec(),
           ConsArgu=CodeConstructor.ConsArgu(),
           ConsInit=CodeConstructor.ConsInit(),
           VarsSpec=CodeConstructor.VarsSpec(),
           VarsArgu=CodeConstructor.VarsArgu(),
           VarsInit=CodeConstructor.VarsInit(),
           InputArguments=CodeConstructor.CppInputArguments(11),
           Initializations=CodeConstructor.CppInitializations(4)))
    lenCons=0
    lenVars=0
    for atom in CodeConstructor.Atoms:
        if atom.constant:
            lenCons+=1
        elif not atom.constant:
            lenVars+=1
    Constructor=CodeConstructor

    for PNOrbitalEvolutionOrder in [frac(n,2) for n in range(0,2*int(HighestOrder)+1)]:
        print("Working on {0} PN...".format(PNOrbitalEvolutionOrder))
        PNOrbitalEvolutionOrderString = str(N(PNOrbitalEvolutionOrder, 2)).replace('.','p')
        %cd -q ../PNTerms
        execnotebook('OrbitalEvolution.ipynb')
        ExpressionsForMemberFunctions = CodeOutput.CodeConstructor(PNVariables, PrecessionVelocities)
        execnotebook('AngularMomentum.ipynb')
        %cd -q -
        PrecessionVelocities.AddDerivedVariable('OrbitalAngularMomentum',
            AngularMomentumExpression(PNOrder=PNOrbitalEvolutionOrder),
            datatype=ellHat.datatype)
        CodeConstructor.AddDependencies(PrecessionVelocities)
        # Start the class, write the declarations, initializations, etc.
        f.write("""
@njit(cache=True)
def Recalculate_{PNOrbitalEvolutionOrderString}(Cons,Vars,y):
    Vars.v = np.array([y[0]])
    Vars.rfrak_chi1 = np.array([y[1],y[2]])
    Vars.rfrak_chi2 = np.array([y[3],y[4]])
    Vars.rfrak_frame = np.array([y[5],y[6],y[7]])
{Evaluations}

{MemberFunctions}
""".format(InputArguments=CodeConstructor.CppInputArguments(11),
           PNOrbitalEvolutionOrderString=PNOrbitalEvolutionOrderString,
           Initializations=CodeConstructor.CppInitializations(4),
           Declarations=CodeConstructor.CppDeclarations(2),
           Evaluations=CodeConstructor.CppEvaluations(4),
           MemberFunctions=CodeOutput.CodeConstructor(PNVariables, PrecessionVelocities).CppExpressionsAsFunctions(2,PNOrbitalEvolutionOrderString)))
        for n,Expressions in zip([1], [T1Expressions]):
            f.write("""
@njit(cache=True)
def TaylorT{n}_{PNOrbitalEvolutionOrderString}(Cons,Vars):
{Computations}
    dydt=np.zeros(8)
    [dydt[5],dydt[6],dydt[7]] = FrameFromAngularVelocityIntegrand(Vars.rfrak_frame, OmegaVec_{PNOrbitalEvolutionOrderString}(Cons,Vars)[1:])
    dydt[0] = dvdt_T{n}[0] 
    if(Cons.EvolveSpin1):
        dydt[1], dydt[2]=FrameFromAngularVelocity_2D_Integrand(Vars.rfrak_chi1[0], Vars.rfrak_chi1[1],(mul(mul(inverse(Cons.S_chi1),OmegaVec_chiVec_1_{PNOrbitalEvolutionOrderString}(Cons,Vars)),Cons.S_chi1))[1:])
    else:
        dydt[1], dydt[2] = 0.0, 0.0
    if(Cons.EvolveSpin2):
        dydt[3], dydt[4]=FrameFromAngularVelocity_2D_Integrand(Vars.rfrak_chi2[0], Vars.rfrak_chi2[1],(mul(mul(inverse(Cons.S_chi2),OmegaVec_chiVec_2_{PNOrbitalEvolutionOrderString}(Cons,Vars)),Cons.S_chi2))[1:])
    else:
        dydt[3], dydt[4] = 0.0, 0.0
    return dydt
""".format(n=n,
           PNOrbitalEvolutionOrderString=PNOrbitalEvolutionOrderString,
           Computations=CodeConstructor.CppEvaluateExpressions()))
        for n,Expressions in zip([4], [T4Expressions]):
            f.write("""
@njit(cache=True)
def TaylorT{n}_{PNOrbitalEvolutionOrderString}(Cons,Vars):
{Computations}
    dydt=np.zeros(8)
    [dydt[5],dydt[6],dydt[7]] = FrameFromAngularVelocityIntegrand(Vars.rfrak_frame, OmegaVec_{PNOrbitalEvolutionOrderString}(Cons,Vars)[1:])
    dydt[0] = dvdt_T{n}[0]
    if(Cons.EvolveSpin1):
        dydt[1], dydt[2]=FrameFromAngularVelocity_2D_Integrand(Vars.rfrak_chi1[0], Vars.rfrak_chi1[1],(mul(mul(inverse(Cons.S_chi1),OmegaVec_chiVec_1_{PNOrbitalEvolutionOrderString}(Cons,Vars)),Cons.S_chi1))[1:])
    else:
        dydt[1], dydt[2] = 0.0, 0.0
    if(Cons.EvolveSpin2):
        dydt[3], dydt[4]=FrameFromAngularVelocity_2D_Integrand(Vars.rfrak_chi2[0], Vars.rfrak_chi2[1],(mul(mul(inverse(Cons.S_chi2),OmegaVec_chiVec_2_{PNOrbitalEvolutionOrderString}(Cons,Vars)),Cons.S_chi2))[1:])
    else:
        dydt[3], dydt[4] = 0.0, 0.0
    return dydt      
""".format(n=n,
           PNOrbitalEvolutionOrderString=PNOrbitalEvolutionOrderString,
           Computations=CodeOutput.CodeConstructor(PNVariables, Expressions).CppEvaluateT45(NumTerms4, DenTerms4, Constructor)))
        for n,Expressions in zip([5], [T5Expressions]):
            f.write("""
@njit(cache=True)
def TaylorT{n}_{PNOrbitalEvolutionOrderString}(Cons,Vars):
{Computations}
    dydt=np.zeros(8)
    [dydt[5],dydt[6],dydt[7]] = FrameFromAngularVelocityIntegrand(Vars.rfrak_frame, OmegaVec_{PNOrbitalEvolutionOrderString}(Cons,Vars)[1:])
    dydt[0] = dvdt_T{n}[0]
    if(Cons.EvolveSpin1):
        dydt[1], dydt[2]=FrameFromAngularVelocity_2D_Integrand(Vars.rfrak_chi1[0], Vars.rfrak_chi1[1],(mul(mul(inverse(Cons.S_chi1),OmegaVec_chiVec_1_{PNOrbitalEvolutionOrderString}(Cons,Vars)),Cons.S_chi1))[1:])
    else:
        dydt[1], dydt[2] = 0.0, 0.0
    if(Cons.EvolveSpin2):
        dydt[3], dydt[4]=FrameFromAngularVelocity_2D_Integrand(Vars.rfrak_chi2[0], Vars.rfrak_chi2[1],(mul(mul(inverse(Cons.S_chi2),OmegaVec_chiVec_2_{PNOrbitalEvolutionOrderString}(Cons,Vars)),Cons.S_chi2))[1:])
    else:
        dydt[3], dydt[4] = 0.0, 0.0
    return dydt      
""".format(n=n,
           PNOrbitalEvolutionOrderString=PNOrbitalEvolutionOrderString,
           Computations=CodeOutput.CodeConstructor(PNVariables, Expressions).CppEvaluateT45(NumTerms5, DenTerms5, Constructor)))

        # Write the external interfaces to the ODE stepper
    Ty=''
    for n,Expressions in zip([1,4,5], [T1Expressions,T4Expressions,T5Expressions]):
        if len(Ty)!=0:
            Ty+=','
        Ty+='\n'+',\n'.join(['            {0}:TaylorT{1}_{2}'.format(i*10+n,n,str(N(i/2, 2)).replace('.','p')) for i in range(1+2*int(HighestOrder))])
    f.write("""
class PNEv:
    def Integrand(t,y):
        PNEv.Recalculate.get(2*PNEv.PNEvolutionOrder)(PNEv.Cons,PNEv.Vars,y)
        dydt=PNEv.Taylor.get(PNEv.TaylorTn+20*PNEv.PNEvolutionOrder)(PNEv.Cons,PNEv.Vars)
        if PNEv.Vars.v>=1.0 and PNEv.NotForward:
            print("Beyond domain of PN validity, this is a good way to terminate.")
            PNEv.terminal1=False
        if dydt[0]<1.0e-12 and PNEv.NotForward:
            print("v is decreasing, which is not an uncommon way to stop.")
            PNEv.terminal2=False
        return dydt
        
    def Evolution(xHat_i, yHat_i, zHat_i, M1_i, M2_i, v_i, S_chi1_i, S_chi2_i, rfrak_frame,
        t_PNStart=False, t_PNEnd=False, PNEvolutionOrder=3.5, TaylorTn=1, StepsPerOrbit=32, ForwardInTime=True, tol=1e-8, MinStep=1e-7): 
        # Initialization of constants
        PNEv.terminal1=True
        PNEv.terminal2=True
        PNEv.NotForward=True
        PNEv.PNEvolutionOrder=PNEvolutionOrder
        PNEv.TaylorTn=TaylorTn
        PNEv.Recalculate={{{ReCal}}}
        PNEv.Taylor={{{Ty}}}
        z=np.array([0.0])
        PNEv.Cons=Cons({ConsInput},True,True)
        PNEv.Vars=Vars({VarsInput})
        Initialization(PNEv.Cons,xHat_i, yHat_i, zHat_i, M1_i, M2_i, v_i, S_chi1_i, S_chi2_i, rfrak_frame)
    
        def terminate(t,y):
            return 1.0*PNEv.terminal1*PNEv.terminal2
        terminate.terminal=True
        TMerger=5.0/(256.0*PNEv.Cons.nu*v_i**8)
        TEnd=TMerger
        if t_PNEnd:
            TEnd=t_PNEnd
        time=[0.0]
        while time[-1]<TEnd and 2*PNEv.Cons.M*(256*PNEv.Cons.nu*(TMerger-time[-1])/5)**(3/8)/StepsPerOrbit>MinStep:
            time.append(time[-1]+(2*PNEv.Cons.M*(256*PNEv.Cons.nu*(TMerger-time[-1])/5)**(3/8)/StepsPerOrbit)[0])
        time=np.delete(time, -1)
       
        # Integrate
        yy=solve_ivp(PNEv.Integrand, [time[0],time[-1]], [v_i,0.0,
            0.0,0.0,0.0,rfrak_frame[0],rfrak_frame[1],rfrak_frame[2]], method='DOP853',
            t_eval=time, dense_output=True, events=terminate, rtol=tol, atol=tol)           
        if ForwardInTime:
            PNEv.NotForward=False
            time=[0.0]
            TStart=-3*TMerger
            if t_PNStart:
                TStart=t_PNStart
            while time[-1]>TStart:
                time.append(time[-1]-(2*PNEv.Cons.M*(256*PNEv.Cons.nu*(TMerger-time[-1])/5)**(3/8)/StepsPerOrbit)[0])
            yyForward=solve_ivp(PNEv.Integrand, [time[0],time[-1]], [v_i,0.0,
                0.0,0.0,0.0,rfrak_frame[0],rfrak_frame[1],rfrak_frame[2]], method='DOP853',
                t_eval=time, dense_output=True, rtol=tol, atol=tol)
            yy.t=np.append(yyForward.t[1:][::-1],yy.t)
            data=np.empty((8,len(yy.t)))
            for i in range(8):
                data[i]=np.append(yyForward.y[i][1:][::-1],yy.y[i])
            yy.y=data
             
        return yy
""".format(ConsInput=','.join(['z' for i in range(lenCons)]),
           VarsInput=','.join(['z' for i in range(lenVars)]),
           InputArguments=CodeConstructor.CppInputArguments(11),
           ReCal=',\n'.join(['            {0}:Recalculate_{1}'.format(i,str(N(i/2, 2)).replace('.','p')) for i in range(1+2*int(HighestOrder))]),
           Ty=Ty,
           PNOrbitalEvolutionOrderString=PNOrbitalEvolutionOrderString))  


        
        
        
# Calculate waveform modes
ellMin=2
ellMax=8
def Express(atom,CodeConstructor):
    A=str(atom)
    for i in CodeConstructor.Atoms:
        try:
            if i.constant:
                j=0
                while j!=-1 and j+len(str(i))<=len(A):
                    j=A.find(str(i),j)
                    if j!=-1:
                        if j+len(str(i)) ==len(A) or not (A[j+len(str(i))].isalnum() or A[j+len(str(i))]=='_'):
                            B=A[0:j]+'Cons.{0}'.format(i)+A[j+len(str(i)):]
                            A=B
                            j=j+5+len(str(i))
                        else:
                            j=j+1
            elif not i.constant:
                j=0
                while j!=-1 and j+len(str(i))<=len(A):
                    j=A.find(str(i),j)
                    if j!=-1:
                        if j+len(str(i)) ==len(A) or not (A[j+len(str(i))].isalnum() or A[j+len(str(i))]=='_'):
                            if j==0:
                                B=A[0:j]+'Vars.{0}'.format(i)+A[j+len(str(i)):]
                                A=B
                                j=j+5+len(str(i))
                            elif not (A[j-1].isalnum() or A[j-1]=='_'):
                                B=A[0:j]+'Vars.{0}'.format(i)+A[j+len(str(i)):]
                                A=B
                                j=j+5+len(str(i))
                            else:
                                j=j+1
                        else:
                            j=j+1
        except:
            pass
    return A

InputArgumentsEvolution=CodeConstructor.CppInputArguments(11)
execnotebook('../PNTerms/WaveformModes.ipynb')
WaveformModeTerms = [WaveformModes_NoSpin, WaveformModes_Spin_Symm, WaveformModes_Spin_Asymm]
for Term in WaveformModeTerms:
    PNVariables.update(Term)
# For some reason I have to both overwrite these variables and pop them...
PNVariables.AddVariable('nHat', constant=True, fundamental=True,
                 substitution_atoms=[], substitution='Quaternions::xHat', datatype='Quaternions.Quaternion');
PNVariables.AddVariable('lambdaHat', constant=True, fundamental=True,
                substitution_atoms=[], substitution='Quaternions::yHat', datatype='Quaternions.Quaternion');
PNVariables.AddVariable('ellHat', constant=True, fundamental=True,
                substitution_atoms=[], substitution='Quaternions::zHat', datatype='Quaternions.Quaternion');
PNVariables.pop(nHat);
PNVariables.pop(lambdaHat);
PNVariables.pop(ellHat);

PNVariables.AddDerivedVariable('chiVec1', substitution_atoms=[chiVec1], substitution='mul(mul(mul(S_chi1,R_S1),zHat),mul(conjugate(R_S1),conjugate(S_chi1)))');
PNVariables.AddDerivedVariable('chiVec2', substitution_atoms=[chiVec2], substitution='mul(mul(mul(S_chi2,R_S2),zHat),mul(conjugate(R_S2),conjugate(S_chi2)))');
PNVariables.AddDerivedVariable('chi1_n', substitution_atoms=[chiVec1,nHat], substitution='np.array([chiVec1[1]])');
PNVariables.AddDerivedVariable('chi1_lambda', substitution_atoms=[chiVec1,lambdaHat], substitution='np.array([chiVec1[2]])');
PNVariables.AddDerivedVariable('chi1_ell', substitution_atoms=[chiVec1,ellHat], substitution='np.array([chiVec1[3]])');
PNVariables.AddDerivedVariable('chi2_n', substitution_atoms=[chiVec2,nHat], substitution='np.array([chiVec2[1]])');
PNVariables.AddDerivedVariable('chi2_lambda', substitution_atoms=[chiVec2,lambdaHat], substitution='np.array([chiVec2[2]])');
PNVariables.AddDerivedVariable('chi2_ell', substitution_atoms=[chiVec2,ellHat], substitution='np.array([chiVec2[3]])');

with open('PNWaveformModes.py', 'w') as f :
    f.write("""# File produced automatically by PNCodeGen.ipynb
import numpy as np
from numpy import dot, log, sqrt, pi
from numpy import euler_gamma as EulerGamma
from numba import jit, njit, float64, complex128
from numba.experimental import jitclass
import quaternionic

qmul = njit(quaternionic.algebra.multiply)
qexp=njit(quaternionic.algebra.exp)
qconj=njit(quaternionic.algebra.conj)
I=1j

@njit(cache=True)
def mul(A,B):
    C=np.empty(4)
    qmul(A,B,C)
    return C
    
@njit(cache=True)
def exp(A):
    B=np.empty(4)
    qexp(A,B)
    return B
    
@njit(cache=True)
def conjugate(A):
    B=np.empty(4)
    qconj(A,B)
    return B
""")

    PNWaveformModeOrder=HighestOrder
    PNWaveformModeOrderString = str(N(PNWaveformModeOrder,2)).replace('.','p')
    ModeExpressions0 = PNCollection()
    LM = [[ell,m] for ell in range(ellMin,ellMax+1) for m in range(-ell,ell+1)]
    for ell in range(ellMin, ellMax+1):
        for m in range(0, ell+1):
            Symm = (SymmetricWaveformModes([ell,m],PNOrder=PNWaveformModeOrder)).subs(log(v), logv)
            Asymm = (AsymmetricWaveformModes([ell,m],PNOrder=PNWaveformModeOrder)).subs(log(v), logv)
            ModeExpressions0.AddDerivedVariable('rhOverM_{0}_{1}_Symm'.format(ell,m),
                                                Symm, datatype='double')
            ModeExpressions0.AddDerivedVariable('rhOverM_{0}_{1}_Asymm'.format(ell,m),
                                               Asymm, datatype='double')
    CodeConstructor0 = CodeOutput.CodeConstructor(PNVariables, ModeExpressions0)
    f.write("""
ConsSpec=[{ConsSpec}]
@jitclass(ConsSpec)
class Constants:
    def __init__(self,{ConsArgu}):
{ConsInit}

VarsSpec=[{VarsSpec}]
@jitclass(VarsSpec)
class Variables:
    def __init__(self,{VarsArgu}):
{VarsInit}

@njit(cache=True)
def Initialization(Cons, {InputArguments}): 
{Initializations}
""".format(ConsSpec=CodeConstructor0.ConsSpec(),
           ConsArgu=CodeConstructor0.ConsArgu(),
           ConsInit=CodeConstructor0.ConsInit(),
           VarsSpec=CodeConstructor0.VarsSpec(),
           VarsArgu=CodeConstructor0.VarsArgu(),
           VarsInit=CodeConstructor0.VarsInit(),
           InputArguments=InputArgumentsEvolution,
           Initializations=CodeConstructor0.CppInitializationModes(4)))
    lenConsReal=0
    lenVarsReal=0
    lenConsComplex=0
    lenVarsComplex=0
    for atom in CodeConstructor0.Atoms:
        if atom.constant and str(atom).find('hHat')==-1:
            lenConsReal+=1
        elif not atom.constant and str(atom).find('hHat')==-1:
            lenVarsReal+=1
        elif atom.constant and str(atom).find('hHat')!=-1:
            lenConsComplex+=1
        elif not atom.constant and str(atom).find('hHat')!=-1:
            lenVarsComplex+=1
    
    for PNWaveformModeOrder in [frac(n,2) for n in range(0,2*int(HighestOrder)+1)]:
        print("Working on {0} PN...".format(PNWaveformModeOrder))
        PNWaveformModeOrderString = str(N(PNWaveformModeOrder,2)).replace('.','p')
        ModeExpressions = PNCollection()                     
        LM = [[ell,m] for ell in range(ellMin,ellMax+1) for m in range(-ell,ell+1)]
        lenLM=len(LM)
        Evaluations = []
        for ell in range(ellMin, ellMax+1):
            for m in range(0, ell+1):
                Symm = (SymmetricWaveformModes([ell,m],PNOrder=PNWaveformModeOrder)).subs(log(v), logv)
                Asymm = (AsymmetricWaveformModes([ell,m],PNOrder=PNWaveformModeOrder)).subs(log(v), logv)
                ModeExpressions.AddDerivedVariable('rhOverM_{0}_{1}_Symm'.format(ell,m),
                                                   Symm, datatype='double')
                ModeExpressions.AddDerivedVariable('rhOverM_{0}_{1}_Asymm'.format(ell,m),
                                                   Asymm, datatype='double')
                code1 = "    Symm = {0}".format(Express(sympy.pycode(Symm),CodeConstructor0))
                code2 = "    Asymm = {0}".format(Express(sympy.pycode(Asymm),CodeConstructor0))
                code3 = "    modes[{2}] = (Symm + Asymm)[0]".format(ell, m, LM.index([ell,m]))
                code4 = "    modes[{2}] = ({3}np.conjugate(Symm - Asymm))[0]".format(ell, -m, LM.index([ell,-m]),
                                                                               ('' if ((ell%2)==0) else '-'))
                try:
                    if int(sympy.pycode(Symm))==0 and int(sympy.pycode(Asymm))==0:
                        pass
                except:
                    Evaluations.append("    # (ell, m) = ({0}, +/- {1})".format(ell, m))
                    Evaluations.append(code1)
                    Evaluations.append(code2)
                    Evaluations.append(code3)
                    if m!=0:
                        Evaluations.append(code4)
        CodeConstructor = CodeOutput.CodeConstructor(PNVariables, ModeExpressions)
        #[print(atom) for atom in CodeConstructor.Atoms if atom.constant]

        # Start the class, write the declarations, initializations, etc.
        f.write("""
@njit(cache=True)
def Recalculate_{PNWaveformModeOrderString}(Cons,Vars,y):
    Vars.v = np.array([y[0]])
    Vars.rfrak_chi1 = np.array([y[1],y[2]])
    Vars.rfrak_chi2 = np.array([y[3],y[4]])
    Vars.rfrak_frame = np.array([y[5],y[6],y[7]])
{Evaluations}

@njit(cache=True)
def Modes_{PNWaveformModeOrderString}(modes,Cons,Vars):
{Computations}
""".format(PNWaveformModeOrderString=PNWaveformModeOrderString,
           Evaluations=CodeConstructor.CppEvaluationModes(4),
           Computations='\n'.join(Evaluations)))
        
    f.write("""
def Modes(xHat_i, yHat_i, zHat_i, M1_i, M2_i, v_i, S_chi1_i, S_chi2_i, rfrak_frame, y, PNWaveformModeOrder=3.5) :
    Recalculate={{{ReCal}}}
    WaveformModes={{{ModeFunction}}}
    ModeData=np.zeros((len(y[0]),{lenLM}), dtype=complex)
    zz=np.array([0.0])
    z=np.array([0.0*1j])
    Cons=Constants({ConsInput})
    Vars=Variables({VarsInput})
    Initialization(Cons, xHat_i, yHat_i, zHat_i, M1_i, M2_i, v_i, S_chi1_i, S_chi2_i, rfrak_frame)
    for i in range(len(y[0])):
        Recalculate.get(2*PNWaveformModeOrder)(Cons,Vars,y[:,i])
        WaveformModes.get(2*PNWaveformModeOrder)(ModeData[i,:],Cons,Vars)
    return ModeData, [{ellMin},{ellMax}]
""".format(ReCal=',\n'.join(['        {0}:Recalculate_{1}'.format(i,str(N(i/2, 2)).replace('.','p')) for i in range(1+2*int(HighestOrder))]),
           ModeFunction=',\n'.join(['        {0}:Modes_{1}'.format(i,str(N(i/2, 2)).replace('.','p')) for i in range(1+2*int(HighestOrder))]),
           ConsInput=','.join(['zz' for i in range(lenConsReal)])+','+','.join(['z' for i in range(lenConsComplex)]),
           VarsInput=','.join(['zz' for i in range(lenVarsReal)])+','+','.join(['z' for i in range(lenVarsComplex)]),
           PNWaveformModeOrderString=PNWaveformModeOrderString,
           InputArgumentsEvolution=InputArgumentsEvolution,
           lenLM=lenLM,
           ellMin=ellMin,
           ellMax=ellMax))

print("All done")

Working on 0 PN...
Working on 1/2 PN...
Working on 1 PN...
Working on 3/2 PN...
Working on 2 PN...
Working on 5/2 PN...
Working on 3 PN...
Working on 7/2 PN...
Working on 4 PN...
Working on 0 PN...
Working on 1/2 PN...
Working on 1 PN...
Working on 3/2 PN...
Working on 2 PN...
Working on 5/2 PN...
Working on 3 PN...
Working on 7/2 PN...
Working on 4 PN...
All done
